In [1]:
# -*- coding: utf-8 -*-
import h5py
import os
import pickle
import numpy as np
import pandas as pd
import argparse
from wang_args import WangClassifier

import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, models
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.metrics.functional.classification import auroc, stat_scores
from sklearn.metrics import RocCurveDisplay, roc_curve, auc, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt


In [10]:
args = argparse.Namespace(mri_sequences='TBAKDEF', data_sequences='TBAKDEF', valid_dir='tbakd3_npy/uint8/5folds/4/train_bal', eval_batch_size=1)
f4 = np.vectorize(lambda x: f'{x:.4f}')
ckpt = '/net/scratch/hanliu/radiology/prostatex/results/wang-fold-bal/z4ktwfw9/checkpoints/epoch=102-valid_acc=0.81.ckpt' # splendid-night-1029
model = WangClassifier.load_from_checkpoint(ckpt, **vars(args))
_ = model.eval()

In [6]:
from lrp.innvestigator import InnvestigateModel


In [11]:
class EnsemblePredict(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        x, y = batch
        x = x[:, self.mri_index]
        logits = self(x)
        y_hat = torch.sigmoid(logits)
        return y_hat.matmul(self.w_ensemble)

In [15]:
model = torch.nn.Sequential(model, EnsemblePredict())
inn_model = InnvestigateModel(model, lrp_exponent=1,
                                  method="e-rule",
                                  beta=0, epsilon=1e-6)
inn_model.eval()

NotImplementedError: The network contains layers that are currently not supported Flatten(start_dim=1, end_dim=-1)